In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
import numpy as np
import qiskit
%matplotlib inline
import random

In [21]:
# Pre-shared key K1 (n+m 0 or 1) and K2 (n+m 0 or 1)

# Step1 : Alice generates SK(distributed session key) (n 0 or 1)
#         Hash(SK||K1)=hsk (m 0 or 1)
#         SK||hsk = Sa (n+m 0 or 1)

def random_01str(list1,length):
    for i in range(length):
        list1.append(random.randint(0,1))
        
n = 8
m = 4

K1=[]
K1.clear()
random_01str(K1,n+m)
print("K1 : {}".format(K1))

K2=[]
K2.clear()
random_01str(K2,n+m)
print("K2 : {}".format(K2))

# Alice

SK=[]
SK.clear()
random_01str(SK,n)
print("SK : {}".format(SK))

SK_K1=[]
SK_K1.clear()
SK_K1.extend(SK+K1)
print("SK_K1 : {}".format(SK_K1))


print((2*n+m)//m) # --> each group's length
# m --> length in Hash

# Hash()
def Hash_preshared(list1,group,length,list2):
    list3=[]
    for i in range(length):
        str3=""
        for j in range(group):
            str3+=f'{list1[group*i+j]}'
        list3.append(str3)
    print("str1 = {}".format(list3))
    
    list5=[]
    for i in range(length):
        list5.append(int(list3[i],2))
    print("str2 = {}".format(list5))
    
    list6=[]
    for i in range(length):
        list6.append((3*int(list3[i],2)+5)%2)
    print("Hstr = {}".format(list6))
    
    list2.extend(list6)
    

        
        

hsk=[]
hsk.clear()
Hash_preshared(SK_K1,(2*n+m)//m,m,hsk)
print("hsk = {}".format(hsk))


Sa=[]
Sa.clear()
Sa.extend(SK+hsk)
print("Sa : {}".format(Sa))

K1 : [1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1]
K2 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
SK : [0, 0, 0, 1, 0, 0, 0, 0]
SK_K1 : [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1]
5
str1 = ['00010', '00011', '00111', '11001']
str2 = [2, 3, 7, 25]
Hstr = [1, 0, 0, 0]
hsk = [1, 0, 0, 0]
Sa : [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]


In [26]:
# Step2 : Alice prepares Ca (n+m 0 or 1), Cb (=Ca)
#         insert Cb (|0> or |1>) into Sa (|0> or |1>) based on K1 (0 left, 1 right) --> Sa'(2n+2m 0 or 1)
#         K3 (2n+2m 0 or 1), K3[i] = K1[i],K2[i], K3 --> I(0),H(1)
#         send Sa' to Bob

Ca=[]
Ca.clear()
random_01str(Ca,n+m)
print("Ca : {}".format(Ca))

Cb=[]
Cb.clear()
Cb.extend(Ca)
print("Cb : {}".format(Cb))


K3=[]
K3.clear()
for i in range(n+m):
    K3.append(K1[i])
    K3.append(K2[i])
    
print("K3 : {}".format(K3))

Ca : [1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1]
Cb : [1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1]
K3 : [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1]


In [30]:
# generate one qubit

q = QuantumRegister(1,'q')
c = ClassicalRegister(2,'c')

# Alice

Alice_circ0 = QuantumCircuit(q,c)  # |0>
Alice_circ0.barrier(q)

Alice_circ1 = QuantumCircuit(q,c)  # |1>
Alice_circ1.x(q[0])
Alice_circ1.barrier(q)


#Alice_circ0.draw()
#Alice_circ1.draw()

K3_I_circ = QuantumCircuit(q,c)  # I operation
K3_I_circ.iden(q[0])
K3_I_circ.barrier(q)

K3_H_circ = QuantumCircuit(q,c)  # H operation
K3_H_circ.h(q[0])
K3_H_circ.barrier(q)

#K3_I_circ.draw()
#K3_H_circ.draw()

In [42]:
# generate circuits

Alice_circ=[Alice_circ0,Alice_circ1]
K3_circ=[K3_I_circ,K3_H_circ]

circuits_Sa2=[]
circuits_Sa2.clear()

for i in range(n+m):
    if K1[i]==0: # Cb , Sa
        circuits_Sa2.append(Alice_circ[Cb[i]] + K3_circ[K3[2*i]])
        circuits_Sa2.append(Alice_circ[Sa[i]] + K3_circ[K3[2*i+1]])
    else: # Sa , Cb
        circuits_Sa2.append(Alice_circ[Sa[i]] + K3_circ[K3[2*i]])
        circuits_Sa2.append(Alice_circ[Cb[i]] + K3_circ[K3[2*i+1]])


print("length of circuits_Sa2 : {}".format(len(circuits_Sa2)))
circuits_Sa2[0].draw()

length of circuits_Sa2 : 24


In [45]:
# Step3 : Bob operates I(0),H(1) based on K3
#         measure Sa to get SK' , hsk'.  resend |0>,|1> to Alice depend on result
#         measure(1) or reflect(0) Cb based on K2.  return opposite state of measurement.
#         (if result of measurement is 0, return |1>.)

circuits_step3_1=[]
circuits_step3_1.clear()

for i in range(n+m):
    if K1[i]==0: # Cb , Sa
        circuits_step3_1.append(circuits_Sa2[2*i] + K3_circ[K3[2*i]])
        circuits_step3_1.append(circuits_Sa2[2*i+1] + K3_circ[K3[2*i+1]])
    else: # Sa , Cb
        circuits_step3_1.append(circuits_Sa2[2*i] + K3_circ[K3[2*i]])
        circuits_step3_1.append(circuits_Sa2[2*i+1] + K3_circ[K3[2*i+1]])


print("length of circuits_step3_1 : {}".format(len(circuits_step3_1)))
circuits_step3_1[0].draw()

length of circuits_step3_1 : 24


In [54]:
# Z measurement

Bob_Z_measurement = QuantumCircuit(q,c) # Z basis
Bob_Z_measurement.measure(q[0],c[0])

# not measure

Bob_not_measure = QuantumCircuit(q,c)


Bob_Z_measurement.draw()
#Bob_not_measure.draw()

In [58]:
Bob_measure_circ=[Bob_not_measure,Bob_Z_measurement]

circuits_step3_Sa=[]
circuits_step3_Sa.clear()

circuits_step3_Cb=[]
circuits_step3_Cb.clear()

for i in range(n+m):
    if K1[i]==0: # Cb , Sa
        circuits_step3_Cb.append(circuits_step3_1[2*i] + Bob_measure_circ[K2[i]])
        circuits_step3_Sa.append(circuits_step3_1[2*i+1] + Bob_Z_measurement)
    else: # Sa , Cb
        circuits_step3_Sa.append(circuits_step3_1[2*i] + Bob_Z_measurement)
        circuits_step3_Cb.append(circuits_step3_1[2*i+1] + Bob_measure_circ[K2[i]])


#print("length of circuits_step3_Sa : {}".format(len(circuits_step3_Sa)))
#circuits_step3_Sa[0].draw()

print("length of circuits_step3_Cb : {}".format(len(circuits_step3_Cb)))
circuits_step3_Cb[10].draw()

length of circuits_step3_Cb : 12


In [59]:
from qiskit import execute, BasicAer
backend_sim = BasicAer.get_backend('qasm_simulator')

job_sim1 = qiskit.execute(circuits_step3_Sa, backend_sim, shots=1024)
result_sim1 = job_sim1.result()
counts1 = result_sim1.get_counts(circuits_step3_Sa[0])
print(counts1)

{'00': 1024}


In [60]:
# get result of Bob measuring Sa

Bob_measure_Sa_result=[]
Bob_measure_Sa_result.clear()

for i in range(n+m):
    Bob_measure_Sa_result.append(int((list(result_sim1.get_counts(circuits_step3_Sa[i]))[0])[1]))
    
print("Bob_measure_Sa_result = {}".format(Bob_measure_Sa_result))

Bob_measure_Sa_result = [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]


In [126]:
# Bob gets SK', hsk'

SK2=[]
SK2.clear()

hsk2=[]
hsk2.clear()

for i in range(n+m):
    if i <n:
        SK2.append(Bob_measure_Sa_result[i])
    else:
        hsk2.append(Bob_measure_Sa_result[i])
        
        
print("SK2 = {}".format(SK2))
print("hsk2 = {}".format(hsk2))

SK2 = [0, 0, 0, 1, 0, 0, 0, 0]
hsk2 = [1, 0, 0, 0]


In [65]:
resend_0_circ = QuantumCircuit(q,c)  # resend |0>
resend_0_circ.reset(q[0])
resend_0_circ.barrier(q)

resend_1_circ = QuantumCircuit(q,c)  # resend |1>
resend_1_circ.reset(q[0])
resend_1_circ.x(q[0])
resend_1_circ.barrier(q)


#resend_0_circ.draw()
resend_1_circ.draw()

In [68]:
Bob_resend_circ=[resend_0_circ,resend_1_circ]

circuits_step3_Sa_resend=[]
circuits_step3_Sa_resend.clear()

for i in range(n+m):
    circuits_step3_Sa_resend.append(circuits_step3_Sa[i] + Bob_resend_circ[Bob_measure_Sa_result[i]])
    
    
print("length of circuits_step3_Sa_resend : {}".format(len(circuits_step3_Sa_resend)))
circuits_step3_Sa_resend[8].draw()

length of circuits_step3_Sa_resend : 12


In [71]:
circuits_step3_Cb_measure=[]
circuits_step3_Cb_measure.clear()

circuits_step3_Cb_reflect=[]
circuits_step3_Cb_reflect.clear()

for i in range(n+m):
    if K2[i]==1:
        circuits_step3_Cb_measure.append(circuits_step3_Cb[i])
    else:
        circuits_step3_Cb_reflect.append(circuits_step3_Cb[i])
        
    
print("length of circuits_step3_Cb_measure : {}".format(len(circuits_step3_Cb_measure)))
print("length of circuits_step3_Cb_reflect : {}".format(len(circuits_step3_Cb_reflect)))

circuits_step3_Cb_measure[0].draw()
#circuits_step3_Cb_reflect[0].draw()

length of circuits_step3_Cb_measure : 2
length of circuits_step3_Cb_reflect : 10


In [72]:
job_sim2 = qiskit.execute(circuits_step3_Cb_measure, backend_sim, shots=1024)
result_sim2 = job_sim2.result()
counts2 = result_sim2.get_counts(circuits_step3_Cb_measure[0])
print(counts2)

{'01': 1024}


In [73]:
# get result of Bob measuring part of Cb

Bob_measure_Cb_result=[]
Bob_measure_Cb_result.clear()

for i in range(len(circuits_step3_Cb_measure)):
    Bob_measure_Cb_result.append(int((list(result_sim2.get_counts(circuits_step3_Cb_measure[i]))[0])[1]))
    
print("Bob_measure_Cb_result = {}".format(Bob_measure_Cb_result))

Bob_measure_Cb_result = [1, 1]


In [76]:
circuits_step3_Cb_resend=[]
circuits_step3_Cb_resend.clear()

for i in range(len(circuits_step3_Cb_measure)):
    circuits_step3_Cb_resend.append(circuits_step3_Cb_measure[i] + Bob_resend_circ[not(Bob_measure_Cb_result[i])])
    
    
print("length of circuits_step3_Cb_resend : {}".format(len(circuits_step3_Cb_resend)))
circuits_step3_Cb_resend[0].draw()

length of circuits_step3_Cb_resend : 2


In [79]:
# prepare to send to Alice, combine Sa and two parts of Cb

circuits_step3_Cb_combine=[]
circuits_step3_Cb_combine.clear()

counter_cb_resend = len(circuits_step3_Cb_resend)
counter_cb_reflect = len(circuits_step3_Cb_reflect)

for i in range(n+m):
    if K2[i]==1:
        circuits_step3_Cb_combine.append(circuits_step3_Cb_resend[len(circuits_step3_Cb_resend)-counter_cb_resend])
        counter_cb_resend=counter_cb_resend-1
    else:
        circuits_step3_Cb_combine.append(circuits_step3_Cb_reflect[len(circuits_step3_Cb_reflect)-counter_cb_reflect])
        counter_cb_reflect=counter_cb_reflect-1
        
    
print("length of circuits_step3_Cb_combine : {}".format(len(circuits_step3_Cb_combine)))

circuits_step3_Cb_combine[11].draw()

length of circuits_step3_Cb_combine : 12


In [89]:
# circuits_step3_Sa_resend + circuits_step3_Cb_combine  
# operates I(0),H(1) based on K3, then send.

circuits_step3_send_to_Alice=[]                            
circuits_step3_send_to_Alice.clear()

for i in range(n+m):
    if K1[i]==0: # Cb , Sa
        circuits_step3_send_to_Alice.append(circuits_step3_Cb_combine[i] + K3_circ[K3[2*i]])
        circuits_step3_send_to_Alice.append(circuits_step3_Sa_resend[i] + K3_circ[K3[2*i+1]])
    else: # Sa , Cb
        circuits_step3_send_to_Alice.append(circuits_step3_Sa_resend[i] + K3_circ[K3[2*i]])
        circuits_step3_send_to_Alice.append(circuits_step3_Cb_combine[i] + K3_circ[K3[2*i+1]])


print("length of circuits_step3_send_to_Alice : {}".format(len(circuits_step3_send_to_Alice)))
circuits_step3_send_to_Alice[0].draw()

length of circuits_step3_send_to_Alice : 24


In [95]:
# Step4 : Alice checks
#         [operates I(0),H(1) based on K3  , then Z measurement] or [Z(0) or X(1) measurement based on K3]

circuits_step4_1=[]
circuits_step4_1.clear()

# Z measurement
Alice_Z_measurement = QuantumCircuit(q,c) # Z basis
Alice_Z_measurement.measure(q[0],c[0])

# X measurement
Alice_X_measurement = QuantumCircuit(q,c) # X basis
Alice_X_measurement.h(q[0])
Alice_X_measurement.measure(q[0],c[0])


Alice_measurement_circ=[Alice_Z_measurement,Alice_X_measurement]


for i in range(n+m):
    if K1[i]==0: # Cb , Sa
        circuits_step4_1.append(circuits_step3_send_to_Alice[2*i] + Alice_measurement_circ[K3[2*i]])     # + K3_circ[K3[2*i]]
        circuits_step4_1.append(circuits_step3_send_to_Alice[2*i+1] + Alice_measurement_circ[K3[2*i+1]]) # + K3_circ[K3[2*i+1]]
    else: # Sa , Cb
        circuits_step4_1.append(circuits_step3_send_to_Alice[2*i] + Alice_measurement_circ[K3[2*i]])
        circuits_step4_1.append(circuits_step3_send_to_Alice[2*i+1] + Alice_measurement_circ[K3[2*i+1]])


print("length of circuits_step4_1 : {}".format(len(circuits_step4_1)))
circuits_step4_1[0].draw()

length of circuits_step4_1 : 24


In [96]:
job_sim3 = qiskit.execute(circuits_step4_1, backend_sim, shots=1024)
result_sim3 = job_sim3.result()
counts3 = result_sim3.get_counts(circuits_step4_1[0])
print(counts3)

{'00': 1024}


In [122]:
# get result of Alice measurement

Alice_measure_result=[]
Alice_measure_result.clear()

for i in range(2*(n+m)):
    Alice_measure_result.append(int((list(result_sim3.get_counts(circuits_step4_1[i]))[0])[1]))
    
    
print("length of Alice_measure_result = {}".format(len(Alice_measure_result)))
print("Alice_measure_result = {}".format(Alice_measure_result))

length of Alice_measure_result = 24
Alice_measure_result = [0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]


In [123]:
Alice_measure_Sa_result=[]
Alice_measure_Sa_result.clear()

Alice_measure_Cb_result=[]
Alice_measure_Cb_result.clear()


for i in range(n+m):
    if K1[i]==0: # Cb , Sa
        Alice_measure_Cb_result.append(Alice_measure_result[2*i])   
        Alice_measure_Sa_result.append(Alice_measure_result[2*i+1])
    else: # Sa , Cb
        Alice_measure_Sa_result.append(Alice_measure_result[2*i])
        Alice_measure_Cb_result.append(Alice_measure_result[2*i+1])


print("length of Alice_measure_Sa_result : {}".format(len(Alice_measure_Sa_result)))
print("Alice_measure_Sa_result : {}".format(Alice_measure_Sa_result))
print("length of Alice_measure_Cb_result : {}".format(len(Alice_measure_Cb_result)))
print("Alice_measure_Cb_result : {}".format(Alice_measure_Cb_result))

length of Alice_measure_Sa_result : 12
Alice_measure_Sa_result : [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]
length of Alice_measure_Cb_result : 12
Alice_measure_Cb_result : [1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0]


In [124]:
# Check Alice_measure_Sa_result
right_times_sa = 0

for i in range(n+m):
    if Alice_measure_Sa_result[i]==Sa[i]:
        right_times_sa+=1
        print("measure Sa {} = origin Sa {}".format(Alice_measure_Sa_result[i],Sa[i]))
    else :
        print("measure Sa {} != origin Sa {}".format(Alice_measure_Sa_result[i],Sa[i]))
        print("Different result has been found.")


print("")
        
error_rate_sa=((n+m-right_times_sa)/(n+m))
print("error rate = {}%".format(error_rate_sa*100))

measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0
measure Sa 1 = origin Sa 1
measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0
measure Sa 1 = origin Sa 1
measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0

error rate = 0.0%


In [125]:
# Check Alice_measure_Cb_result
right_times_cb = 0

for i in range(n+m):
    if K2[i]==0:
        if Alice_measure_Cb_result[i]==Ca[i]:
            right_times_cb+=1
            print("measure Cb {} = origin Ca {} --> right".format(Alice_measure_Cb_result[i],Ca[i]))
        else:
            print("measure Cb {} != origin Ca {} --> wrong".format(Alice_measure_Cb_result[i],Ca[i]))
    else:
        if Alice_measure_Cb_result[i]!=Ca[i]:
            right_times_cb+=1
            print("measure Cb {} != origin Ca {} --> right".format(Alice_measure_Cb_result[i],Ca[i]))
        else:
            print("measure Cb {} = origin Ca {} --> wrong".format(Alice_measure_Cb_result[i],Ca[i]))
           

        
print("") 
        
error_rate_cb=((n+m-right_times_cb)/(n+m))
print("error rate = {}%".format(error_rate_cb*100))

measure Cb 1 = origin Ca 1 --> right
measure Cb 1 = origin Ca 1 --> right
measure Cb 0 = origin Ca 0 --> right
measure Cb 0 = origin Ca 0 --> right
measure Cb 1 = origin Ca 1 --> right
measure Cb 0 = origin Ca 0 --> right
measure Cb 0 = origin Ca 0 --> right
measure Cb 1 = origin Ca 1 --> right
measure Cb 0 = origin Ca 0 --> right
measure Cb 1 = origin Ca 1 --> right
measure Cb 0 != origin Ca 1 --> right
measure Cb 0 != origin Ca 1 --> right

error rate = 0.0%


In [128]:
# Step4 : meanwhile, Bob checks H(SK'||K1)==hsk' and get key

print("SK2 = {}".format(SK2))
print("hsk2 = {}".format(hsk2))
print("Above are Bob's measurement result")
print("")

SK2_K1=[]
SK2_K1.clear()
SK2_K1.extend(SK2+K1)
print("SK2_K1 : {}".format(SK2_K1))

#(2*n+m)//m --> each group's length
# m --> length in Hash

Hash_SK2_K1=[]
Hash_SK2_K1.clear()
Hash_preshared(SK2_K1,(2*n+m)//m,m,Hash_SK2_K1)
print("Hash_SK2_K1 = {}".format(Hash_SK2_K1))

SK2 = [0, 0, 0, 1, 0, 0, 0, 0]
hsk2 = [1, 0, 0, 0]
Above are Bob's measurement result

SK2_K1 : [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1]
str1 = ['00010', '00011', '00111', '11001']
str2 = [2, 3, 7, 25]
Hstr = [1, 0, 0, 0]
Hash_SK2_K1 = [1, 0, 0, 0]


In [129]:
# Bob checks H(SK'||K1)==hsk'
right_times_hsk = 0

for i in range(m):
    if Hash_SK2_K1[i]==hsk2[i]:
        right_times_hsk+=1
        print("Hash_SK2_K1 {} = hsk2 {}".format(Hash_SK2_K1[i],hsk2[i]))
    else :
        print("Hash_SK2_K1 {} != hsk2 {}".format(Hash_SK2_K1[i],hsk2[i]))
        print("Different result has been found.")


print("")
        
if right_times_hsk==m:
    print("H(SK'||K1) == hsk'")
else:
    print("H(SK'||K1) != hsk'")

Hash_SK2_K1 1 = hsk2 1
Hash_SK2_K1 0 = hsk2 0
Hash_SK2_K1 0 = hsk2 0
Hash_SK2_K1 0 = hsk2 0

H(SK'||K1) == hsk'


In [130]:
# If no Eve, Bob gets key
print("SK2 = {}".format(SK2))

SK2 = [0, 0, 0, 1, 0, 0, 0, 0]


In [150]:
# eavesdropper

# Eve measurement
E1 = QuantumCircuit(q,c) # Z basis
E1.measure(q[0],c[1])

E2 = QuantumCircuit(q,c) # X basis
E2.h(q[0])
E2.measure(q[0],c[1])

E3 = QuantumCircuit(q,c) # don't measure


def EveRandom_012str(list,num):
    for i in range(num):
        list.append(random.randint(0,2))

        
# Eve between step2 and step3
Eve_basis_step23=[]
Eve_basis_step23.clear()
EveRandom_012str(Eve_basis_step23,(2*n+2*m))
print("Eve_basis_step23   E1(0)(Z) or E2(1)(X) or E3(2)(don't measure) : {}".format(Eve_basis_step23))
print(len(Eve_basis_step23))


# Eve between step3 and step4
Eve_basis_step34=[]
Eve_basis_step34.clear()
EveRandom_012str(Eve_basis_step34,(2*n+2*m))
print("Eve_basis_step34   E1(0)(Z) or E2(1)(X) or E3(2)(don't measure) : {}".format(Eve_basis_step34))
print(len(Eve_basis_step34))


Eve_circ = [E1,E2,E3]
# + Eve_circ[Eve_basis_step23[i]] , + Eve_circ[Eve_basis_step34[i]]

Eve_basis_step23   E1(0)(Z) or E2(1)(X) or E3(2)(don't measure) : [0, 2, 2, 1, 2, 0, 0, 1, 0, 1, 2, 1, 1, 1, 1, 0, 1, 2, 0, 1, 2, 1, 0, 0]
24
Eve_basis_step34   E1(0)(Z) or E2(1)(X) or E3(2)(don't measure) : [0, 0, 1, 1, 0, 2, 0, 1, 0, 2, 0, 1, 0, 2, 2, 0, 1, 1, 2, 0, 2, 0, 1, 2]
24


In [151]:
# if Eve measures between step2 and step3

circuits_step3_1_eve23=[]
circuits_step3_1_eve23.clear()

for i in range(n+m):
    if K1[i]==0: # Cb , Sa
        circuits_step3_1_eve23.append(circuits_Sa2[2*i] + Eve_circ[Eve_basis_step23[2*i]] + K3_circ[K3[2*i]])
        circuits_step3_1_eve23.append(circuits_Sa2[2*i+1] + Eve_circ[Eve_basis_step23[2*i+1]] + K3_circ[K3[2*i+1]])
    else: # Sa , Cb
        circuits_step3_1_eve23.append(circuits_Sa2[2*i] + Eve_circ[Eve_basis_step23[2*i]] + K3_circ[K3[2*i]])
        circuits_step3_1_eve23.append(circuits_Sa2[2*i+1] + Eve_circ[Eve_basis_step23[2*i+1]] + K3_circ[K3[2*i+1]])


print("length of circuits_step3_1_eve23 : {}".format(len(circuits_step3_1_eve23)))
circuits_step3_1_eve23[0].draw()

length of circuits_step3_1_eve23 : 24


In [152]:
circuits_step3_Sa_eve23=[]
circuits_step3_Sa_eve23.clear()

circuits_step3_Cb_eve23=[]
circuits_step3_Cb_eve23.clear()

for i in range(n+m):
    if K1[i]==0: # Cb , Sa
        circuits_step3_Cb_eve23.append(circuits_step3_1_eve23[2*i] + Bob_measure_circ[K2[i]])
        circuits_step3_Sa_eve23.append(circuits_step3_1_eve23[2*i+1] + Bob_Z_measurement)
    else: # Sa , Cb
        circuits_step3_Sa_eve23.append(circuits_step3_1_eve23[2*i] + Bob_Z_measurement)
        circuits_step3_Cb_eve23.append(circuits_step3_1_eve23[2*i+1] + Bob_measure_circ[K2[i]])


print("length of circuits_step3_Sa_eve23 : {}".format(len(circuits_step3_Sa_eve23)))

print("length of circuits_step3_Cb_eve23 : {}".format(len(circuits_step3_Cb_eve23)))

circuits_step3_Sa_eve23[0].draw()
#circuits_step3_Cb_eve23[10].draw()

length of circuits_step3_Sa_eve23 : 12
length of circuits_step3_Cb_eve23 : 12


In [153]:
job_sim1_eve23 = qiskit.execute(circuits_step3_Sa_eve23, backend_sim, shots=1024)
result_sim1_eve23 = job_sim1_eve23.result()
counts1_eve23 = result_sim1_eve23.get_counts(circuits_step3_Sa_eve23[0])
print(counts1_eve23)

{'00': 259, '11': 258, '01': 247, '10': 260}


In [154]:
# get result of Bob measuring Sa

Bob_measure_Sa_result_eve23=[]
Bob_measure_Sa_result_eve23.clear()

for i in range(n+m):
    Bob_measure_Sa_result_eve23.append(int((list(result_sim1_eve23.get_counts(circuits_step3_Sa_eve23[i]))[0])[1]))
    
print("Bob_measure_Sa_result_eve23 = {}".format(Bob_measure_Sa_result_eve23))

Bob_measure_Sa_result_eve23 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [155]:
# Bob gets SK', hsk'

SK2_eve23=[]
SK2_eve23.clear()

hsk2_eve23=[]
hsk2_eve23.clear()

for i in range(n+m):
    if i <n:
        SK2_eve23.append(Bob_measure_Sa_result_eve23[i])
    else:
        hsk2_eve23.append(Bob_measure_Sa_result_eve23[i])
        
        
print("SK2_eve23 = {}".format(SK2_eve23))
print("hsk2_eve23 = {}".format(hsk2_eve23))

SK2_eve23 = [0, 0, 0, 0, 0, 0, 0, 0]
hsk2_eve23 = [0, 0, 0, 0]


In [156]:
# Bob checks H(SK'||K1)==hsk' and get key

print("SK2_eve23 = {}".format(SK2_eve23))
print("hsk2_eve23 = {}".format(hsk2_eve23))
print("Above are Bob's measurement result")
print("")

SK2_K1_eve23=[]
SK2_K1_eve23.clear()
SK2_K1_eve23.extend(SK2_eve23+K1)
print("SK2_K1_eve23 : {}".format(SK2_K1_eve23))

#(2*n+m)//m --> each group's length
# m --> length in Hash

Hash_SK2_K1_eve23=[]
Hash_SK2_K1_eve23.clear()
Hash_preshared(SK2_K1_eve23,(2*n+m)//m,m,Hash_SK2_K1_eve23)
print("Hash_SK2_K1_eve23 = {}".format(Hash_SK2_K1_eve23))

SK2_eve23 = [0, 0, 0, 0, 0, 0, 0, 0]
hsk2_eve23 = [0, 0, 0, 0]
Above are Bob's measurement result

SK2_K1_eve23 : [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1]
str1 = ['00000', '00011', '00111', '11001']
str2 = [0, 3, 7, 25]
Hstr = [1, 0, 0, 0]
Hash_SK2_K1_eve23 = [1, 0, 0, 0]


In [157]:
# Bob checks H(SK'||K1)==hsk'
right_times_hsk_eve23 = 0

for i in range(m):
    if Hash_SK2_K1_eve23[i]==hsk2_eve23[i]:
        right_times_hsk_eve23+=1
        print("Hash_SK2_K1 {} = hsk2 {}".format(Hash_SK2_K1_eve23[i],hsk2_eve23[i]))
    else :
        print("Hash_SK2_K1 {} != hsk2 {}".format(Hash_SK2_K1_eve23[i],hsk2_eve23[i]))
        print("Different result has been found.")


print("")
        
if right_times_hsk_eve23==m:
    print("H(SK'||K1) == hsk'")
else:
    print("H(SK'||K1) != hsk'")

Hash_SK2_K1 1 != hsk2 0
Different result has been found.
Hash_SK2_K1 0 = hsk2 0
Hash_SK2_K1 0 = hsk2 0
Hash_SK2_K1 0 = hsk2 0

H(SK'||K1) != hsk'


In [158]:
# if Eve measures between step3 and step4

circuits_step4_1_eve34=[]
circuits_step4_1_eve34.clear()


for i in range(n+m):
    if K1[i]==0: # Cb , Sa
        circuits_step4_1_eve34.append(circuits_step3_send_to_Alice[2*i] + Eve_circ[Eve_basis_step34[2*i]] + Alice_measurement_circ[K3[2*i]])     # + K3_circ[K3[2*i]]
        circuits_step4_1_eve34.append(circuits_step3_send_to_Alice[2*i+1] + Eve_circ[Eve_basis_step34[2*i+1]] + Alice_measurement_circ[K3[2*i+1]]) # + K3_circ[K3[2*i+1]]
    else: # Sa , Cb
        circuits_step4_1_eve34.append(circuits_step3_send_to_Alice[2*i] + Eve_circ[Eve_basis_step34[2*i]] + Alice_measurement_circ[K3[2*i]])
        circuits_step4_1_eve34.append(circuits_step3_send_to_Alice[2*i+1] + Eve_circ[Eve_basis_step34[2*i+1]] + Alice_measurement_circ[K3[2*i+1]])


print("length of circuits_step4_1_eve34 : {}".format(len(circuits_step4_1_eve34)))
circuits_step4_1_eve34[2].draw()

length of circuits_step4_1_eve34 : 24


In [159]:
job_sim3_eve34 = qiskit.execute(circuits_step4_1_eve34, backend_sim, shots=1024)
result_sim3_eve34 = job_sim3_eve34.result()
counts3_eve34 = result_sim3_eve34.get_counts(circuits_step4_1_eve34[0])
print(counts3_eve34)

{'00': 281, '11': 235, '01': 255, '10': 253}


In [160]:
# get result of Alice measurement

Alice_measure_result_eve34=[]
Alice_measure_result_eve34.clear()

for i in range(2*(n+m)):
    Alice_measure_result_eve34.append(int((list(result_sim3_eve34.get_counts(circuits_step4_1_eve34[i]))[0])[1]))
    
    
print("length of Alice_measure_result_eve34 = {}".format(len(Alice_measure_result_eve34)))
print("Alice_measure_result_eve34 = {}".format(Alice_measure_result_eve34))

length of Alice_measure_result_eve34 = 24
Alice_measure_result_eve34 = [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]


In [161]:
Alice_measure_Sa_result_eve34=[]
Alice_measure_Sa_result_eve34.clear()

Alice_measure_Cb_result_eve34=[]
Alice_measure_Cb_result_eve34.clear()


for i in range(n+m):
    if K1[i]==0: # Cb , Sa
        Alice_measure_Cb_result_eve34.append(Alice_measure_result_eve34[2*i])   
        Alice_measure_Sa_result_eve34.append(Alice_measure_result_eve34[2*i+1])
    else: # Sa , Cb
        Alice_measure_Sa_result_eve34.append(Alice_measure_result_eve34[2*i])
        Alice_measure_Cb_result_eve34.append(Alice_measure_result_eve34[2*i+1])


print("length of Alice_measure_Sa_result_eve34 : {}".format(len(Alice_measure_Sa_result_eve34)))
print("Alice_measure_Sa_result_eve34 : {}".format(Alice_measure_Sa_result_eve34))
print("length of Alice_measure_Cb_result_eve34 : {}".format(len(Alice_measure_Cb_result_eve34)))
print("Alice_measure_Cb_result_eve34 : {}".format(Alice_measure_Cb_result_eve34))

length of Alice_measure_Sa_result_eve34 : 12
Alice_measure_Sa_result_eve34 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
length of Alice_measure_Cb_result_eve34 : 12
Alice_measure_Cb_result_eve34 : [1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0]


In [162]:
# Check Alice_measure_Sa_result_eve34
right_times_sa_eve34 = 0

for i in range(n+m):
    if Alice_measure_Sa_result_eve34[i]==Sa[i]:
        right_times_sa_eve34+=1
        print("measure Sa {} = origin Sa {}".format(Alice_measure_Sa_result_eve34[i],Sa[i]))
    else :
        print("measure Sa {} != origin Sa {}".format(Alice_measure_Sa_result_eve34[i],Sa[i]))
        print("Different result has been found.")


print("")
        
error_rate_sa_eve34=((n+m-right_times_sa_eve34)/(n+m))
print("error rate = {}%".format(error_rate_sa_eve34*100))

measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0
measure Sa 0 != origin Sa 1
Different result has been found.
measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0
measure Sa 0 != origin Sa 1
Different result has been found.
measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0
measure Sa 0 = origin Sa 0

error rate = 16.666666666666664%


In [163]:
# Check Alice_measure_Cb_result
right_times_cb_eve34 = 0

for i in range(n+m):
    if K2[i]==0:
        if Alice_measure_Cb_result_eve34[i]==Ca[i]:
            right_times_cb_eve34+=1
            print("measure Cb {} = origin Ca {} --> right".format(Alice_measure_Cb_result_eve34[i],Ca[i]))
        else:
            print("measure Cb {} != origin Ca {} --> wrong".format(Alice_measure_Cb_result_eve34[i],Ca[i]))
    else:
        if Alice_measure_Cb_result_eve34[i]!=Ca[i]:
            right_times_cb_eve34+=1
            print("measure Cb {} != origin Ca {} --> right".format(Alice_measure_Cb_result_eve34[i],Ca[i]))
        else:
            print("measure Cb {} = origin Ca {} --> wrong".format(Alice_measure_Cb_result_eve34[i],Ca[i]))
           

        
print("") 
        
error_rate_cb_eve34=((n+m-right_times_cb_eve34)/(n+m))
print("error rate = {}%".format(error_rate_cb_eve34*100))

measure Cb 1 = origin Ca 1 --> right
measure Cb 0 != origin Ca 1 --> wrong
measure Cb 0 = origin Ca 0 --> right
measure Cb 0 = origin Ca 0 --> right
measure Cb 1 = origin Ca 1 --> right
measure Cb 0 = origin Ca 0 --> right
measure Cb 0 = origin Ca 0 --> right
measure Cb 1 = origin Ca 1 --> right
measure Cb 0 = origin Ca 0 --> right
measure Cb 1 = origin Ca 1 --> right
measure Cb 0 != origin Ca 1 --> right
measure Cb 0 != origin Ca 1 --> right

error rate = 8.333333333333332%
